In [106]:
import pandas as pd
import numpy as np
import random
import ast
import itertools

In [107]:
class Encoder():

  def __init__(self, values_to_encode=None):
    self.encoding_dict = None
    self.decoding_dict = None
    if values_to_encode: self.build_encodings(values_to_encode)

  def build_encodings(self, values):
    self.encoding_dict = {v: i for i, v in enumerate(values)}
    self.decoding_dict = {i: v for i, v in enumerate(values)}

  def encode(self, value):
    return self.encoding_dict[value]

  def decode(self, value):
    return self.decoding_dict[value]

In [108]:
class Graph():

  def __init__(self, graph_filepath=None):
    self.edges = None
    self.place_encoder = None
    self.distance_matrix = None
    if graph_filepath: self.load_graph_from_excel(graph_filepath)

  def load_graph_from_excel(self, filepath, sheet_name="grafo"):
    self.edges = pd.read_excel(filepath, sheet_name=sheet_name)
    unique_places = list(set(
        list(self.edges["edge 1"].values) +
        list(self.edges["edge 2"].values)
    ))
    n_unique_places = len(unique_places)
    self.place_encoder = Encoder(list(unique_places))
    self.distance_matrix = np.full((n_unique_places, n_unique_places), np.inf)
    for _, row in self.edges.iterrows():
      (
        self.distance_matrix
        [self.place_encoder.encode(row["edge 1"])]
        [self.place_encoder.encode(row["edge 2"])]
      ) = row["distance"]
    self.distance_matrix[np.eye(n_unique_places, dtype="bool")] = 0
    self.__reduce_distance_matrix()

  def __reduce_distance_matrix(self):
    n = self.distance_matrix.shape[0]
    for k in range(n):
        for i in range(n):
            for j in range(n):
                self.distance_matrix[i][j] = min(
                    self.distance_matrix[i][j],
                    self.distance_matrix[i][k] + self.distance_matrix[k][j]
                )

In [109]:
class Solver():
    
  def __init__(self, graph, product_query,
               start_place_name="Casa", fuel_cost_per_distance=.1, catalog_filepath=None):
    self.graph = graph
    self.hub_id = self.graph.place_encoder.encode(start_place_name)
    self.product_query = product_query
    self.fuel_cost = fuel_cost_per_distance
    self.catalog = None
    self.product_encoder = None
    #self.store_encoder = None
    self.price_matrix = None
    self.distance_matrix = None
    if catalog_filepath: self.load_catalog_from_excel(catalog_filepath)

  def load_catalog_from_excel(self, filepath, sheet_name="catalogo"):
    self.catalog = pd.read_excel(filepath, sheet_name=sheet_name)
    self.product_encoder = Encoder(list(self.catalog["product"].unique()))
    #self.store_encoder = Encoder(list(self.catalog["store"].unique()))
    self.price_matrix = (
      self.catalog
      .pivot(index="product", columns="store", values="price")
      .rename(index = self.product_encoder.encode, columns = self.graph.place_encoder.encode)
      .sort_index(axis=0)
      .sort_index(axis=1)
      #.to_numpy()
    )


In [110]:
class GeneticSolver(Solver):

  def __initialize_population(self, size):
    store_ids = list(self.price_matrix.columns)
    product_ids = [self.product_encoder.encode(i) for i in self.product_query]
    return [[(random.choice(store_ids), product) for product in product_ids] for i in range(size)]

  def __get_total_product_cost(self, gene):
    return sum([self.price_matrix[store][product] for store, product in gene])

  def __get_total_travel_cost(self, gene):
    travel_route = list(zip(
        [self.hub_id] + [store for store, _ in gene],
        [store for store, _ in gene] + [self.hub_id]
    ))
    return sum([self.graph.distance_matrix[store_a][store_b] * self.fuel_cost for store_a, store_b in travel_route])

  def evaluate_solution(self, gene):
    return self.__fitness(gene)

  def prettify_gene(self, gene):
    store_ids, product_ids = list(zip(*gene))
    stores = [self.graph.place_encoder.decode(s) for s in store_ids]
    products = [self.product_encoder.decode(p) for p in product_ids]
    return list(zip(stores, products))

  def __fitness(self, gene, replace_nan_with=np.nan):
    fitness_value = self.__get_total_product_cost(gene) + self.__get_total_travel_cost(gene)
    if np.isnan(fitness_value): fitness_value = replace_nan_with
    return fitness_value

  def solve(self, n_iters=10, pop_size=100, selection_rate=0.5, mutation_prob=0.1):
    population = self.__initialize_population(pop_size)
    selection_size = int(pop_size * selection_rate)
    children_amount = pop_size - selection_size
    for _ in range(n_iters):
      best_individuals = self.__selection(population, selection_size)
      children = self.__crossover(best_individuals, children_amount)
      #children = self.__mutation(children, mutation_prob)
      population = best_individuals + children
    return self.__selection(population, 1)[0]

  def __selection(self, genes, selection_size):
    return sorted(genes, key = lambda x: self.__fitness(x, replace_nan_with=np.inf))[:selection_size]

  def __crossover(self, genes, children_amount):
    children = []
    while len(children) < children_amount:
      parent_1 = random.choice(genes)
      parent_2 = random.choice(genes)
      p1_stores, p1_products = list(zip(*parent_1))
      p2_stores, p2_products = list(zip(*parent_2))
      product_crossover_children = self.__partially_mapped_crossover(p1_products, p2_products)
      store_crossover_children = self.__one_point_crossover(p1_stores, p2_stores)
      child_1 = list(zip(store_crossover_children[0], product_crossover_children[0]))
      child_2 = list(zip(store_crossover_children[1], product_crossover_children[1]))
      children += [child_1, child_2]
    return children[:children_amount]

  def __one_point_crossover(self, parent_1, parent_2, cross_point=None):
    if cross_point is None: cross_point = random.randint(0, len(parent_1)-1)
    child_1 = parent_1[:cross_point] + parent_2[cross_point:]
    child_2 = parent_2[:cross_point] + parent_1[cross_point:]
    return child_1, child_2

  def __partially_mapped_crossover(self, parent_1, parent_2, cross_point=None):
    if cross_point is None: cross_point = random.randint(0, len(parent_1)-1)
    mappings = {a: b for a, b in zip(parent_1[:cross_point], parent_2[:cross_point])}
    mappings.update({b: a for a, b in zip(parent_1[:cross_point], parent_2[:cross_point])})
    child_1 = parent_1[:cross_point] + tuple(mappings.get(i, i) for i in parent_2[cross_point:])
    child_2 = parent_2[:cross_point] + tuple(mappings.get(i, i) for i in parent_1[cross_point:])
    return child_1, child_2, mappings

  def __mutation(self, genes, mutation_prob):
    for gene in genes:
      if random.random() < mutation_prob:
        gene = self.__swap_mutation(gene)
    return genes

  def __swap_mutation(self, gene):
     idx = range(len(gene))
     i1, i2 = random.sample(idx, 2)
     gene[i1], gene[i2] = gene[i2], gene[i1]
     return gene



In [111]:
class GreedyCheapest(GeneticSolver):

    """
    Prioritizes cheapest products for the user
    """

    def solve(self):

        product_ids = [self.product_encoder.encode(i) for i in self.product_query]
        solution = []
        for i, j in self.price_matrix.loc[product_ids, :].iterrows():
            solution.append((self.price_matrix.columns[j.argmin()], i))
        solution.sort(key = lambda x : x[0])

        return solution

In [112]:
class GreedyNearest(GeneticSolver):

    """
    Prioritizes supermarkets closer to the user
    """

    def solve(self):

        nearest_place = self.graph.place_encoder.encoding_dict["Casa"]
        product_ids = [self.product_encoder.encode(i) for i in self.product_query]

        # Preprocess distance matrix
        tdm = self.graph.distance_matrix.copy()
        place_ids = list(self.graph.place_encoder.encoding_dict.values())
        store_ids = list(self.price_matrix.columns)
        for p in place_ids:
            if p not in store_ids:
                tdm[:, p] = np.inf
                
        solution = []
        explored_products = set()

        while len(solution) != len(self.product_query):
            tdm[:, nearest_place] = np.inf
            nearest_place = np.argmin(tdm[nearest_place])
            products = self.price_matrix.loc[:, nearest_place]
            products = [i for i in products[products.notnull()].index if i not in explored_products and i in product_ids]
            explored_products.update(products)
            solution += zip([nearest_place] * len(products), products)

        return solution

In [113]:
class BruteForceSolver(GeneticSolver):

    """
    This solver searches all possible solutions
    """

    def _backtrack(self, combination, index):
        
        if index == len(self.product_query):
            return [combination[:]]

        results = []
        for store in self.price_matrix.columns:
            product = self.price_matrix.index[index]
            if not pd.isna(self.price_matrix.loc[product, store]):
                combination.append((store, product))
                results += self._backtrack(combination, index + 1)
                combination.pop()
        
        return results
    
    def _evaluate_bf_solutions(self, store_combinations, catalog_combinations):

        best_val = np.inf
        best_comb = None

        for place_order in store_combinations:
            for comb in catalog_combinations:
                # Reorder comb
                new_comb = []
                for order in place_order:
                    new_comb += [i for i in comb if i[0] == order]
                # Evaluate and update
                val = self.evaluate_solution(new_comb)
                if val != np.nan and val < best_val:
                    best_val = val
                    best_comb = new_comb
        
        return best_comb
    
    def _init_solve(self):

        store_ids = list(self.price_matrix.columns)
        product_ids = [self.product_encoder.encode(i) for i in self.product_query]
        self.price_matrix = self.price_matrix.loc[product_ids, :]

        #print("Building catalog combinations...")
        #catalog_combinations = list(list(zip(element, product_ids))
            #for element in itertools.product(store_ids, repeat = len(product_ids)))
        catalog_combinations = self._backtrack([], 0)
        
        #print("Building store combinations...")
        store_combinations = list(itertools.permutations(store_ids))

        return catalog_combinations, store_combinations

    def solve(self):

        store_combinations, catalog_combinations = self._init_solve()

        #print("Getting best value...")
        best_comb = self._evaluate_bf_solutions(store_combinations, catalog_combinations)

        return best_comb

In [114]:
class QuasiBruteForceSolver(BruteForceSolver):

    """
    This solver searches a sample of all possible solutions
    """

    def solve(self, max_catalog_comb_sample=10000, max_store_comb_sample=1000):

        catalog_combinations, store_combinations = self._init_solve()
        catalog_combinations = random.sample(catalog_combinations,
                            min(len(catalog_combinations), max_catalog_comb_sample))
        store_combinations = random.sample(store_combinations,
                            min(len(store_combinations), max_store_comb_sample))

        best_comb = self._evaluate_bf_solutions(store_combinations, catalog_combinations)

        return best_comb
    

In [115]:
class GeneticPurchase():

    SUCURSALES = [
        "Supermercado Nacional",
        "Jumbo",
        "La Sirena",
        "Super Pola",
        "Bravo",
        "Super Fresh Market",
        "Olé",
        "Supermercado Carrefour",
        "Supermercado Iberia",
        "Supermercado El Encanto",
        "Supermercado Aprecio",
        "Supermercado Garrido",
        "Supermercado La Fuente",
        "Supermercado La Cadena",
        "Supermercado Jacaranda",
        "Supermercado Bella Vista Mall",
        "Supermercado Super Lama",
        "Supermercado Amigo",
        "Supermercado Los Hidalgos",
        "Supermercado Central",
        "Supermercado Hiper Uno",
        "Supermercado Plaza Lama",
        "Supermercado El Escogido",
        "Supermercado Mr. Food",
        "Supermercado Los Prados",
        "Supermercado Tropical",
        "Supermercado Ever",
        "Supermercado Yaguar",
        "Supermercado Precio Justo",
        "Supermercado El Ahorrón",
        "Supermercado Distribuidora Corripio",
        "Supermercado Plaza Comercial",
        "Supermercado Las Colinas",
        "Supermercado La Fuente del Norte",
        "Supermercado Plaza Real",
        "Supermercado Bravo Oriental",
        "Supermercado Supermercado Nuñez",
        "Supermercado Central Dominicano",
        "Supermercado El Sol",
        "Supermercado Plaza Tania",
        "Supermercado El Buen Precio",
        "Supermercado Sabana Grande",
        "Supermercado Lupo",
        "Supermercado La Cadena Oriental",
        "Supermercado Las Américas",
        "Supermercado Megacentro",
        "Supermercado El Tropical",
        "Supermercado La Plaza",
        "Supermercado Super Lopez",
        "Supermercado Los Jardines",
        "Supermercado Super U",
        "Supermercado El Mirador",
        "Supermercado Super Plaza",
        "Supermercado Market Center",
        "Supermercado Plaza Caracol",
        "Supermercado Super Selectos",
        "Supermercado Villa Juana",
        "Supermercado Bravo Norte",
        "Supermercado La Fé",
        "Supermercado El Farolito",
        "Supermercado Villa Francisca",
        "Supermercado Super Martínez",
        "Supermercado Caribe",
        "Supermercado La Espiga",
        "Supermercado Super Max",
        "Supermercado Hipermercado Plaza",
        "Supermercado Gran Maravilla",
        "Supermercado Super Selecto",
        "Supermercado La Preferida",
        "Supermercado Santa Fe",
        "Supermercado El Éxito",
        "Supermercado El Bienvenido",
        "Supermercado El Progreso",
        "Supermercado La Perla",
        "Supermercado El Buen Pastor",
        "Supermercado El Imperio",
        "Supermercado Super Fresh",
        "Supermercado El Oriente",
        "Supermercado Super Norte",
        "Supermercado El Faro",
        "Supermercado El Milagro",
        "Supermercado Super La Fuente",
        "Supermercado El Paraiso",
        "Supermercado Plaza de la Salud",
        "Supermercado Super Rey",
        "Supermercado Plaza Bella",
        "Supermercado El Rancho",
        "Supermercado La Cumbre",
        "Supermercado La Preferencia",
        "Supermercado Plaza Oriental",
        "Supermercado Super La Amistad",
        "Supermercado El Poderoso",
        "Supermercado La Nueva Esperanza",
        "Supermercado Super Plaza Norte",
        "Supermercado La Esperanza",
        "Supermercado Super La Rosa",
        "Supermercado Super Donde Sea",
        "Supermercado La Nueva",
        "Supermercado El Almacen",
        "Supermercado La Economía"
    ]
    PRODUCTOS = [
        "Leche entera",
        "Leche descremada",
        "Leche sin lactosa",
        "Huevos",
        "Pan de molde",
        "Pan integral",
        "Pan baguette",
        "Mantequilla",
        "Margarina",
        "Queso cheddar",
        "Queso mozzarella",
        "Queso parmesano",
        "Yogur natural",
        "Yogur de fresa",
        "Yogur griego",
        "Jamón de pavo",
        "Jamón de cerdo",
        "Salami",
        "Tocino",
        "Pollo entero",
        "Pechugas de pollo",
        "Muslos de pollo",
        "Carne molida de res",
        "Filete de res",
        "Chuletas de cerdo",
        "Costillas de cerdo",
        "Pescado tilapia",
        "Pescado salmón",
        "Camarones",
        "Atún enlatado",
        "Sardinas enlatadas",
        "Aceite de oliva",
        "Aceite de girasol",
        "Aceite de coco",
        "Arroz blanco",
        "Arroz integral",
        "Pasta espagueti",
        "Pasta penne",
        "Pasta fusilli",
        "Harina de trigo",
        "Harina de maíz",
        "Azúcar blanca",
        "Azúcar morena",
        "Sal de mesa",
        "Sal marina",
        "Pimienta negra",
        "Pimienta blanca",
        "Orégano",
        "Albahaca",
        "Perejil",
        "Cilantro",
        "Tomates",
        "Cebollas",
        "Ajo",
        "Pimientos verdes",
        "Pimientos rojos",
        "Zanahorias",
        "Lechuga",
        "Espinacas",
        "Brócoli",
        "Coliflor",
        "Calabacín",
        "Pepinos",
        "Berenjenas",
        "Manzanas",
        "Peras",
        "Plátanos",
        "Naranjas",
        "Limones",
        "Fresas",
        "Arándanos",
        "Mango",
        "Piña",
        "Uvas",
        "Sandía",
        "Melón",
        "Cereal de maíz",
        "Cereal de avena",
        "Granola",
        "Miel",
        "Mermelada de fresa",
        "Mermelada de durazno",
        "Café molido",
        "Café instantáneo",
        "Té negro",
        "Té verde",
        "Jugo de naranja",
        "Jugo de manzana",
        "Refresco de cola",
        "Refresco de limón",
        "Agua mineral",
        "Agua purificada",
        "Galletas de chocolate",
        "Galletas de vainilla",
        "Galletas saladas",
        "Chocolate negro",
        "Chocolate con leche",
        "Chicles de menta",
        "Chicles de fresa",
        "Palomitas de maíz",
        "Papas fritas",
        "Nachos",
        "Salsa de tomate",
        "Salsa barbacoa",
        "Mayonesa",
        "Mostaza",
        "Ketchup",
        "Vino tinto",
        "Vino blanco",
        "Cerveza",
        "Vodka",
        "Ron",
        "Whisky",
        "Gin",
        "Champaña",
        "Papel higiénico",
        "Papel toalla",
        "Servilletas",
        "Jabón de manos",
        "Jabón de baño",
        "Shampoo",
        "Acondicionador",
        "Pasta de dientes",
        "Cepillo de dientes",
        "Hilo dental",
        "Desodorante",
        "Cremas hidratantes",
        "Protector solar",
        "Detergente para ropa",
        "Suavizante de telas",
        "Limpiador multiusos",
        "Esponjas de cocina",
        "Bolsas de basura",
        "Fósforos",
        "Encendedores",
        "Velas",
        "Linterna",
        "Pilas AA",
        "Pilas AAA",
        "Cables USB",
        "Adaptadores de corriente",
        "Cargadores de teléfono",
        "Fundas para celular",
        "Audífonos",
        "Altavoces Bluetooth",
        "Televisores",
        "Reproductores de DVD",
        "Consolas de videojuegos",
        "Juegos de mesa",
        "Libros de cocina",
        "Revistas de moda",
        "Juguetes para niños",
        "Pañales",
        "Toallitas húmedas",
        "Biberones",
        "Chupetes",
        "Ropa de bebé",
        "Zapatos de bebé",
        "Ropa de hombre",
        "Ropa de mujer",
        "Zapatos de hombre",
        "Zapatos de mujer",
        "Bolígrafos",
        "Lápices",
        "Cuadernos",
        "Libretas",
        "Carpetas",
        "Tijeras",
        "Pegamento",
        "Cinta adhesiva",
        "Gomas de borrar",
        "Reglas",
        "Calculadoras",
        "Mochilas",
        "Bolsos",
        "Maletas",
        "Sombreros",
        "Gafas de sol",
        "Paraguas",
        "Cinturones",
        "Billeteras",
        "Relojes",
        "Joyas",
        "Cadenas",
        "Pulseras",
        "Anillos",
        "Collares",
        "Aretes",
        "Perfumes",
        "Colonia",
        "Loción",
        "Aftershave",
        "Mascarillas faciales",
        "Espejos",
        "Secadores de cabello",
        "Planchas de pelo",
        "Rizadores de pelo",
        "Maquillaje",
        "Lápiz labial",
        "Sombra de ojos",
        "Rímel",
        "Base de maquillaje",
        "Esmalte de uñas",
        "Removedor de esmalte",
        "Cortaúñas",
        "Limas de uñas",
        "Cintas para el cabello",
        "Cepillos para el cabello",
        "Peines",
        "Brochas de maquillaje",
        "Toallas",
        "Sábanas",
        "Fundas de almohada",
        "Edredones",
        "Colchas",
        "Almohadas",
        "Cobijas",
        "Cortinas",
        "Persianas",
        "Alfombras",
        "Tapetes",
        "Lamparas de mesa",
        "Lamparas de techo",
        "Ventiladores",
        "Aire acondicionado",
        "Calefactores",
        "Humidificadores",
        "Deshumidificadores",
        "Purificadores de aire",
        "Filtros de agua",
        "Desinfectante de manos",
        "Jabón líquido",
        "Toallas sanitarias",
        "Tampones",
        "Copas menstruales",
        "Protectores diarios",
        "Champú para bebés",
        "Jabón para bebés",
        "Loción para bebés",
        "Aceite para bebés",
        "Cremas para pañales",
        "Termómetros",
        "Gasas",
        "Curitas",
        "Alcohol",
        "Agua oxigenada",
        "Algodón",
        "Palitos de algodón",
        "Vendas",
        "Analgésicos",
        "Antiinflamatorios",
        "Antihistamínicos",
        "Antisépticos",
        "Jarabe para la tos",
        "Vitaminas",
        "Suplementos",
        "Galletas de avena",
        "Galletas de mantequilla",
        "Bizcochos",
        "Pasteles",
        "Donas",
        "Muffins",
        "Croissants",
        "Tartas",
        "Brownies",
        "Helado de vainilla",
        "Helado de chocolate",
        "Helado de fresa",
        "Helado de menta",
        "Helado de café",
        "Conos de helado",
        "Tortillas de maíz",
        "Tortillas de harina",
        "Pan pita",
        "Pan naan",
        "Pan árabe",
        "Arepas",
        "Enchiladas",
        "Quesadillas",
        "Nachos con queso",
        "Salsa de guacamole",
        "Salsa picante",
        "Salsa ranchera",
        "Salsa de soja",
        "Salsa teriyaki",
        "Salsa de ostras",
        "Salsa de pescado",
        "Vinagre de manzana",
        "Vinagre balsámico",
        "Vinagre blanco",
        "Aceitunas verdes",
        "Aceitunas negras",
        "Pepinillos",
        "Alcaparras",
        "Almendras",
        "Nueces",
        "Avellanas",
        "Cacahuates",
        "Pistachos",
        "Anacardos",
        "Semillas de girasol",
        "Semillas de calabaza",
        "Semillas de chía"
    ]
    
    @staticmethod
    def generate_random_gp(sucursales, productos, lugares=None, min_price=100, max_price=5000, min_dist=0.1, max_dist=20, n_cat_elim=0, n_graph_elim=0):
        
        if lugares is None:
            lugares = sucursales

        catalog = pd.concat([
            pd.DataFrame(itertools.product(sucursales, productos), columns=["store", "product"]),
            pd.DataFrame(np.random.randint(min_price, max_price, len(productos)*len(sucursales)) + 0.99, columns=["price"])
        ], axis=1)

        graph = pd.concat([
            pd.DataFrame(itertools.product(lugares, lugares), columns=["edge 1", "edge 2"]),
            pd.DataFrame(np.random.randint(min_dist, max_dist, len(lugares)*len(lugares)), columns=["distance"])
        ], axis=1)

        return catalog.sample(len(catalog)-n_cat_elim), graph.sample(len(graph)-n_graph_elim)
    
    @staticmethod
    def generate_problemset_cluster(n_sucursales, n_productos, low_q, high_q, q_step, low_fp, high_fp, fp_step, cluster_name, folder_prefix="problemset/in", **kwargs):
        
        problems = []
        sucursales = GeneticPurchase.SUCURSALES[:n_sucursales]
        productos = random.sample(GeneticPurchase.PRODUCTOS, n_productos)
        catalog, graph = GeneticPurchase.generate_random_gp(sucursales, productos, lugares=sucursales+["Casa"], **kwargs)
        excel_filename = folder_prefix + f"/{cluster_name}_catalog.xlsx"

        with pd.ExcelWriter(excel_filename) as writer:
            catalog.to_excel(writer, sheet_name="catalogo", index=False)
            graph.to_excel(writer, sheet_name="grafo", index=False)

        for fp in range(low_fp, high_fp+1, fp_step):
            for n_query in range(low_q, high_q+1, q_step):
                query = random.sample(productos, n_query)
                problems.append(pd.Series([cluster_name, excel_filename, fp, query]))
        df_problems = pd.concat(problems, axis=1).T
        df_problems.columns = ["problem_cluster", "catalog", "fuel_price", "query"]
        
        return df_problems

        

In [116]:
if False:
    problemset = pd.concat([
        GeneticPurchase.generate_problemset_cluster(3, 4, 1, 4, 3, 1, 10, 5, "small-3s-4p-f1_10-q1_4-random"),
        GeneticPurchase.generate_problemset_cluster(5, 10, 1, 10, 3, 0, 10, 3, "small-5s-10p-f0_10-q1_10-random", n_cat_elim=10, n_graph_elim=5),
        GeneticPurchase.generate_problemset_cluster(5, 10, 1, 10, 3, 0, 10, 3, "small-5s-10p-f0_10-q1_10-same_price", n_cat_elim=10, n_graph_elim=5, min_price=100, max_price=101),
        GeneticPurchase.generate_problemset_cluster(5, 10, 1, 10, 3, 0, 10, 3, "small-5s-10p-f0_10-q1_10-same_dist", n_cat_elim=10, n_graph_elim=5, min_dist=10, max_dist=11),
        GeneticPurchase.generate_problemset_cluster(5, 10, 1, 10, 3, 0, 10, 3, "small-5s-10p-f0_10-q1_10-jumbocheap", n_cat_elim=10, n_graph_elim=5), # AJUSTADO MANUAL
        GeneticPurchase.generate_problemset_cluster(5, 10, 1, 10, 3, 0, 10, 3, "small-5s-10p-f0_10-q1_10-parque", n_cat_elim=10, n_graph_elim=5), # AJUSTADO MANUAL
        GeneticPurchase.generate_problemset_cluster(5, 10, 1, 10, 3, 0, 10, 3, "small-5s-10p-f0_10-q1_10-punico", n_cat_elim=10, n_graph_elim=5), # AJUSTADO MANUAL
        GeneticPurchase.generate_problemset_cluster(20, 50, 1, 20, 5, 1, 10, 4, "medium-20s-50p-f0_10-q1_20-random", n_cat_elim=200, n_graph_elim=200),
        GeneticPurchase.generate_problemset_cluster(30, 80, 1, 20, 5, 1, 10, 4, "medium-30s-80p-f0_10-q1_20-random", n_cat_elim=500, n_graph_elim=500),
        GeneticPurchase.generate_problemset_cluster(100, 300, 1, 20, 5, 1, 10, 4, "big-100s-300p-f0_10-q1_20-random", n_cat_elim=5000, n_graph_elim=1000),
    ])
    problemset.to_excel("raw_problemset.xlsx")

In [117]:
import pandas as pd
problemset = pd.read_excel("raw_problemset.xlsx")
problemset

,Unnamed: 0,problem_cluster,catalog,fuel_price,query
0,0,small-3s-4p-f1_10-q1_4-random,problemset/in/small-3s-4p-f1_10-q1_4-random_ca...,1,['Purificadores de aire']
1,1,small-3s-4p-f1_10-q1_4-random,problemset/in/small-3s-4p-f1_10-q1_4-random_ca...,1,"['Purificadores de aire', 'Aceite de girasol',..."
2,2,small-3s-4p-f1_10-q1_4-random,problemset/in/small-3s-4p-f1_10-q1_4-random_ca...,6,['Purificadores de aire']
3,3,small-3s-4p-f1_10-q1_4-random,problemset/in/small-3s-4p-f1_10-q1_4-random_ca...,6,"['Purificadores de aire', 'Pasta de dientes', ..."
4,0,small-5s-10p-f0_10-q1_10-random,problemset/in/small-5s-10p-f0_10-q1_10-random_...,0,['Televisores']
...,...,...,...,...,...
131,7,big-100s-300p-f0_10-q1_20-random,problemset/in/big-100s-300p-f0_10-q1_20-random...,5,"['Gasas', 'Alcohol', 'Té negro', 'Tijeras', 'T..."
132,8,big-100s-300p-f0_10-q1_20-random,problemset/in/big-100s-300p-f0_10-q1_20-random...,9,['Detergente para ropa']
133,9,big-100s-300p-f0_10-q1_20-random,problemset/in/big-100s-300p-f0_10-q1_20-random...,9,"['Mermelada de durazno', 'Pañales', 'Leche ent..."
134,10,big-100s-300p-f0_10-q1_20-random,problemset/in/big-100s-300p-f0_10-q1_20-random...,9,"['Galletas de chocolate', 'Pechugas de pollo',..."


In [118]:
problem = problemset.iloc[104]
graph = Graph(graph_filepath=problem["catalog"])
fuel_price = problem["fuel_price"]
query = ast.literal_eval(problem["query"])
problem

Unnamed: 0                                                         4
problem_cluster                    medium-20s-50p-f0_10-q1_20-random
catalog            problemset/in/medium-20s-50p-f0_10-q1_20-rando...
fuel_price                                                         5
query                                       ['Semillas de calabaza']
Name: 104, dtype: object

In [120]:
_solver = GeneticSolver(graph, query, catalog_filepath=problem["catalog"],fuel_cost_per_distance=fuel_price)
result = _solver.solve()
print(_solver.prettify_gene(result))
print(_solver.evaluate_solution(result))

[('Bravo', 'Semillas de calabaza')]
238.99


In [ ]:
_solver = QuasiBruteForceSolver(graph, query, catalog_filepath=problem["catalog"],fuel_cost_per_distance=fuel_price)
result = _solver.solve()
print(_solver.prettify_gene(result))
print(_solver.evaluate_solution(result))

In [121]:
_solver = GreedyNearest(graph, query, catalog_filepath=problem["catalog"],fuel_cost_per_distance=fuel_price)
result = _solver.solve()
print(_solver.prettify_gene(result))
print(_solver.evaluate_solution(result))

[('Supermercado El Encanto', 'Semillas de calabaza')]
3822.99


In [122]:
_solver = GreedyCheapest(graph, query, catalog_filepath=problem["catalog"],fuel_cost_per_distance=fuel_price)
result = _solver.solve()
print(_solver.prettify_gene(result))
print(_solver.evaluate_solution(result))

[('Bravo', 'Semillas de calabaza')]
238.99
